In [1]:
from ltlcross_wrapper import Modulizer
from tools import sem_pipelines
from owl_servers import OwlServerManager, get_ldba_pipeline

We use the client-server mode of Owl. This prevents Owl to start JVM for each command separately. Instead, we start one server for each configuration. These servers listen on ports (in our case) 6060 and 6061 which we then use to call these servers using `owl-client` command. The scripts to start and terminate servers, together with scripts to configure Owl, are in [owl_servers.py](owl_servers.py).

For each configuration we actually run 2 runs: One with Owl alone (`no.owl#variant`), the other (`yes.owl#variant`) uses Spot to simplfy the results using `autfilt`.

In [2]:
owl_m = OwlServerManager()
owl_m.new_server("owl#s", get_ldba_pipeline(mode="s"))
owl_m.new_server("owl#a", get_ldba_pipeline(mode="a"))

owl_pipelines = {}
for mode, cmd in owl_m.get_clients().items():
    owl_pipelines[f"yes.{mode}"] = f"{cmd} | autfilt --small --tgba > %O"
    owl_pipelines[f"no.{mode}"] = f"{cmd} > %O"

We take the Seminator configurations from the file [tools.py](tools.py). The printed values of `tools` are of the form `name_of_the_configuration : ltlcross_command`. This dict is then passed to `Modulizer` from `ltlcross_wrapper` for parallel execution of `ltlcross`.

In [3]:
tools = sem_pipelines.copy()
tools.update(owl_pipelines)
for name, cmd in tools.items():
    print(f"{name}: {cmd}")

yes.seminator#def: ltl2tgba --deterministic -f %f | seminator  --postprocess=1  > %O
no.seminator#def: ltl2tgba --deterministic -f %f | seminator  --postprocess=0  > %O
yes.seminator#tgba: ltl2tgba --deterministic -f %f | seminator --via-tgba --postprocess=1  > %O
no.seminator#tgba: ltl2tgba --deterministic -f %f | seminator --via-tba --postprocess=0  > %O
yes.seminator#tba: ltl2tgba --deterministic -f %f | seminator --via-tgba --postprocess=1  > %O
no.seminator#tba: ltl2tgba --deterministic -f %f | seminator --via-tba --postprocess=0  > %O
yes.seminator#sba: ltl2tgba --deterministic -f %f | seminator --via-sba --postprocess=1  > %O
no.seminator#sba: ltl2tgba --deterministic -f %f | seminator --via-sba --postprocess=0  > %O
yes.seminator-1-1: ltl2tgba --deterministic -f %f | seminator-1.1    > %O
no.seminator-1-1: ltl2tgba --deterministic -f %f | seminator-1.1  -s0  > %O
yes.seminator-1-2: ltl2tgba --deterministic -f %f | seminator-1.2    > %O
no.seminator-1-2: ltl2tgba --deterministic

In [4]:
!mkdir -p data
for s in ["literature","random"]:
    m = Modulizer(tools, f"../formulae/{s}_nd.ltl", 
                  name=f"{s}_nd", tmp_dir=f"data/{s}_nd.parts",
                  processes=6, timeout="30")
    m.run()
    !mv {s}_nd* data

In [5]:
for s in ["literature","random"]:
    for t in ["det","sd"]:
        name = f"{s}_{t}"
        m = Modulizer(tools, f"../formulae/{name}.ltl", 
                      name=name, tmp_dir=f"data/{name}.parts",
                      processes=6, timeout="30")
        m.run()
        !mv {name}* data

In [6]:
owl_m.terminate_all()